In [31]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [32]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5 cm-cr-review-stars-spacing-big'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [33]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=grocery&i=pantry&srs=9574332031&crid=3BD2GKOTQMU4N&sprefix=gr%2Cpantry%2C222&ref=nb_sb_ss_pltr-sample-20_1_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

C:\Users\rohan\AppData\Local\Temp\ipykernel_25544\3235002540.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  amazon_df['title'].replace('', np.nan, inplace=True)


In [34]:
amazon_df

,title,price,rating,reviews,availability
3,ITC Aashirvaad Atta with Multi Grains 11lb,,4.4 out of 5 stars,"14,996 ratings",Not Available
11,Amazon Brand - Vedaka Desi Cow Ghee with Rich ...,,4.1 out of 5 stars,"1,079 ratings",Not Available
12,"Aashirvaad Svasti Pure Cow Ghee, 1L PET",,4.3 out of 5 stars,"3,677 ratings",Not Available
13,Sunfeast Dark Fantasy Coffee Fills,,4.4 out of 5 stars,"2,563 ratings",Not Available
15,Red Chilly Flakes 300Gm (10.58 Oz ),,4.4 out of 5 stars,"1,186 ratings",In Stock
19,"Madhur Pure and Hygienic Sugar, 1kg Bag",,4.4 out of 5 stars,"5,941 ratings",Not Available
28,"Sunfeast Dark Fantasy Choco Fills, 75g (Buy 3 ...",,4.4 out of 5 stars,"6,097 ratings",Not Available
29,Premium Californian Dried and Sweet Whole Cran...,,4.2 out of 5 stars,699 ratings,Not Available
